In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt  
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as fm
import matplotlib.patches as patch
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Introducción.

- En este notebook, encontraremos lineas de código para analizar los juegos de la Máquina Cemetera del Cruz Azul.
    - Las fuentes de datos es un archivo csv que he extraje de Fivethirtyeight y al cual añadí los disparos totales con ayuda de Fbref.com
    - Para utilizarlo pueden descargar el archivo a su equipo o usar la liga que proporcionar en las primeras líneas de comandos

In [36]:
# liga donde obtendremos los datos del archivo

link = 'https://raw.githubusercontent.com/EduardoSanchezZ/Proyecto-Sports-Analytics/main/Data/Cruz%20Azul/Juegos_de_Cruz_Azul.csv'
data = pd.read_csv(link)
data[data['DT']=='Raul Gutierrez'].head(1)

,ID_Partido,Ciclo,Torneo,ID_Torneo,Fecha,Fase,Condicion,DT,Rival,Pts_pronosticados,Goles_anotados,Goles_recibidos,Disparos_realizados,Disparos_permitidos,xGF,xGC,xJG,xJE,xJP
215,216,2022-23,A-22,12,2022-08-27,REG,Local,Raul Gutierrez,Querétaro,1.84,2,1,8,11,1.04,1.9,0.19,0.23,0.59


# Variables del dataset

    - ID_Partido: Número de partido de la base de datos
    - Ciclo: Año futbolístico (temporada)
    - Torneo: Nombre del torneo
    - Fecha: Fecha del juego
    - Fase: Regular o Liguilla
    - Condición: Local o visitante
    - DT: Director Técnico
    - Rival: Equipo al que se enfrentan 
    - Pts_pronosticados: Proyección de Fivethirtyeight
    - Goles_anotados: Goles anotados en el partido
    - Goles_recibidos: Goles recibidos en el partido
    - Disparos_realizados: Disparos realizados por jugadores de Cruz Azul
    - Disparos_permitidos: Disparos realizados por jugadores rivales
    - xGF: Probabilidad acumulada de que los disparos de Cruz Azul terminen en gol 
    - xGC: Probabilidad acumulada de que los disparos rivales terminen en gol
    - xJG, xJE, xJP: Haciendo una simulación, calculamos la probabilidad de que Cruz Azul gané, empate o pierda

# Agrupación por DT
    - Uno de los análisis más frecuentes es el analizar el desempeño del equipo desglosado por Director Técnico

In [40]:
# Creamos un dataframe nuevo

df = data.copy()

# Agregamos variables para victorias, derrotas, empates y puntos

df['jg'] = np.where(df['Goles_anotados']>df['Goles_recibidos'],1,0)
df['je'] = np.where(df['Goles_anotados']==df['Goles_recibidos'],1,0)
df['jp'] = np.where(df['Goles_anotados']<df['Goles_recibidos'],1,0)
df['pts'] = np.where(df['Goles_anotados']>df['Goles_recibidos'],3,np.where(df['Goles_anotados']<df['Goles_recibidos'],0,1))
df['xpts'] = df['xJG'] * 3 + df['xJE']

# Agrupamos por Director Técnico (DT), debido a que hay diferentes muestras de juegos, utilizaremos medias

dt_agg = df.groupby(['DT','Torneo','ID_Torneo']).agg(juegos = ('Fecha', 'count'),
                              exp_jg = ('xJG', 'sum'),
                              exp_je = ('xJE', 'sum'),
                              exp_jp = ('xJP', 'sum'),
                              exp_pts_prom = ('xpts', 'mean'),
                              ganados = ('jg', 'sum'),
                              empates = ('je', 'sum'),
                              derrotas = ('jp', 'sum'),
                              pts_prom = ('pts', 'mean')).reset_index().sort_values(by=['ID_Torneo','juegos'], ascending=[True,False])

dt_agg['exp_jg'] = round(dt_agg['exp_jg'],1)
dt_agg['exp_je'] = round(dt_agg['exp_je'],1)
dt_agg['exp_jp'] = round(dt_agg['exp_jp'],1)
dt_agg['exp_pts_prom'] = round(dt_agg['exp_pts_prom'],3)
dt_agg['pts_prom'] = round(dt_agg['pts_prom'],3)

dt_agg = dt_agg.reset_index(drop=True)

dt_agg.index = dt_agg.index + 1

#Output

# Mensaje 
print("Desempeño de Cruz Azul, desglosado por Director Téçnico:")

dt_agg.drop('ID_Torneo', axis=1)

Desempeño de Cruz Azul, desglosado por Director Téçnico:


,DT,Torneo,juegos,exp_jg,exp_je,exp_jp,exp_pts_prom,ganados,empates,derrotas,pts_prom
1,Francisco Jemez,C-17,17,7.6,5.4,4.0,1.655,5,6,6,1.235
2,Francisco Jemez,A-17,19,8.4,5.1,5.5,1.595,7,8,4,1.526
3,Pedro Caixinha,C-18,17,7.6,4.2,5.3,1.582,5,7,5,1.294
4,Pedro Caixinha,A-18,23,7.6,6.5,8.9,1.269,13,5,5,1.913
5,Pedro Caixinha,C-19,19,9.2,4.9,4.9,1.707,9,6,4,1.737
6,Robert Siboldi,A-19,10,4.4,2.8,2.8,1.605,3,4,3,1.300
7,Pedro Caixinha,A-19,8,3.9,2.0,2.1,1.712,2,4,2,1.250
8,Robert Siboldi,C-20,10,4.5,2.1,3.4,1.551,7,1,2,2.200
9,Robert Siboldi,G-20,21,9.5,4.6,7.0,1.574,11,2,8,1.667
10,Juan Reynoso,G-21,23,9.5,6.6,6.9,1.529,16,4,3,2.261
